In [1]:
import csv
from random import seed, randrange
from math import sqrt
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from scipy.stats import ttest_rel

# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Convert string column to float where possible, otherwise convert to int
def preprocess_dataset(dataset):
    for i in range(len(dataset[0]) - 1):  # Assuming last column is the class label
        try:
            # Attempt to convert each value in the column to float
            column = [row[i] for row in dataset]
            float_column = list(map(float, column))
            for j, row in enumerate(dataset):
                row[i] = float_column[j]
        except ValueError:
            # On failure, assume column is categorical and encode it
            encoder = LabelEncoder()
            column = [row[i] for row in dataset]
            int_column = encoder.fit_transform(column)
            for j, row in enumerate(dataset):
                row[i] = int_column[j]
    
    # Encode class label column
    class_labels = [row[-1] for row in dataset]
    encoder = LabelEncoder()
    int_class_labels = encoder.fit_transform(class_labels)
    for j, row in enumerate(dataset):
        row[-1] = int_class_labels[j]

# Normalize dataset
def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])-1):
        col_values = [row[i] for row in dataset]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax

def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):  # Exclude label
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

# Euclidean Distance Calculation
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i]) ** 2
    return sqrt(distance)

# Get Neighbors
def get_neighbors(train, test_row, num_neighbors):
    distances = list()
    for train_row in train:
        dist = euclidean_distance(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = [distances[i][0] for i in range(num_neighbors)]
    return neighbors

# Make a Prediction
def predict_classification(train, test_row, num_neighbors):
    neighbors = get_neighbors(train, test_row, num_neighbors)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
    return prediction

# KNN Algorithm
def k_nearest_neighbors(train, test, num_neighbors):
    predictions = list()
    for row in test:
        output = predict_classification(train, row, num_neighbors)
        predictions.append(output)
    return predictions

# Cross Validation Split
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

# Load dataset
filename = 'breast-cancer.data'  # Update this path as necessary
dataset = load_csv(filename)
preprocess_dataset(dataset)  # Preprocess to handle categorical and numeric data
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

# Evaluate KNN
n_folds = 10
num_neighbors = 5
scores = evaluate_algorithm(dataset, k_nearest_neighbors, n_folds, num_neighbors)

# Sklearn KNN for comparison
data = [row[:-1] for row in dataset]
labels = [row[-1] for row in dataset]
model = KNeighborsClassifier(n_neighbors=num_neighbors)
cv = KFold(n_splits=n_folds, random_state=1, shuffle=True)
sklearn_scores = cross_val_score(model, data, labels, scoring='accuracy', cv=cv, n_jobs=-1)

# Paired t-test
t_stat, p_value = ttest_rel(scores, sklearn_scores)
print(f'Custom KNN Scores: {scores}')
print('Custom KNN Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
print(f'Scikit-learn KNN Scores: {sklearn_scores}')
print('Scikit-learn KNN Mean Accuracy: %.3f%%' % (sklearn_scores.mean() * 100))
print(f'Paired t-test t-statistic: {t_stat}, p-value: {p_value}')


alpha = 0.05
print("Value of alpha:- ",alpha)
if p_value > alpha:
    print('Fails to reject H0 (Null Hypothesis)')
else:
    print('Reject H0 (Null Hypothesis)')


Custom KNN Scores: [71.42857142857143, 78.57142857142857, 75.0, 85.71428571428571, 71.42857142857143, 85.71428571428571, 71.42857142857143, 60.71428571428571, 78.57142857142857, 78.57142857142857]
Custom KNN Mean Accuracy: 75.714%
Scikit-learn KNN Scores: [0.86206897 0.68965517 0.68965517 0.75862069 0.75862069 0.79310345
 0.78571429 0.57142857 0.85714286 0.85714286]
Scikit-learn KNN Mean Accuracy: 76.232%
Paired t-test t-statistic: 31.826121960129836, p-value: 1.4658847137706711e-10
Value of alpha:-  0.05
Reject H0 (Null Hypothesis)


In [2]:
#Calculations using Manhattan Distance

from random import seed, randrange
from csv import reader
from math import sqrt

# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Convert string column to float and handle ranges
def convert_range_to_numeric(dataset, column_indices):
    for row in dataset:
        for index in column_indices:
            range_parts = row[index].split('-')
            if len(range_parts) == 2:
                numeric_value = (float(range_parts[0]) + float(range_parts[1])) / 2
            else:
                numeric_value = float(range_parts[0])
            row[index] = numeric_value

# Convert string column to integer for categorical data
def encode_categorical_data(dataset):
    categorical_columns = [i for i in range(len(dataset[0])) if isinstance(dataset[0][i], str)]
    for column in categorical_columns:
        class_values = [row[column] for row in dataset]
        unique = set(class_values)
        lookup = dict()
        for i, value in enumerate(unique):
            lookup[value] = i
        for row in dataset:
            row[column] = lookup[row[column]]

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate Manhattan distance between two vectors
def manhattan_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += abs(row1[i] - row2[i])
    return distance

# Calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)

# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors, distance_function):
    distances = list()
    for train_row in train:
        dist = distance_function(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors

# Make a prediction with neighbors
def predict_classification(train, test_row, num_neighbors, distance_function):
    neighbors = get_neighbors(train, test_row, num_neighbors, distance_function)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
    return prediction

# K-Nearest Neighbors Algorithm
def k_nearest_neighbors(train, test, num_neighbors, distance_function):
    predictions = list()
    for row in test:
        output = predict_classification(train, row, num_neighbors, distance_function)
        predictions.append(output)
    return predictions

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, num_neighbors, distance_function):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, num_neighbors, distance_function)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Main execution
if __name__ == "__main__":
    seed(1)
    filename = 'breast-cancer.data'  
    dataset = load_csv(filename)
    convert_range_to_numeric(dataset, [1, 3])
    encode_categorical_data(dataset)
    n_folds = 10
    num_neighbors = 5
    # Choose the distance function: manhattan_distance or euclidean_distance
    distance_function = manhattan_distance  # or euclidean_distance
    scores = evaluate_algorithm(dataset, k_nearest_neighbors, n_folds, num_neighbors, distance_function)
    print('Scores:', scores)
    print('Mean Accuracy using Manhattan Distance:', sum(scores)/float(len(scores)))


Scores: [78.57142857142857, 67.85714285714286, 60.71428571428571, 64.28571428571429, 75.0, 92.85714285714286, 85.71428571428571, 75.0, 78.57142857142857, 67.85714285714286]
Mean Accuracy using Manhattan Distance: 74.64285714285714
